In [25]:
import numpy as np


class Martingale:
    def __init__(self, open_price, max_cnt, direction, bs, margin_call, close_call):
        self.max_cnt = max_cnt  # 最大补仓次数
        self.direction = direction  # 方向 buy 多 sell 空
        self.bs = bs  # 补仓数量 like: [1, 2, 4, 8]...
        self.margin_call = margin_call  # 跌 | 涨 1% 补仓
        self.close_call = close_call  # 获利多少平仓
        self.curr = 0  # 当前开仓次数

        self.price_lists = np.array([open_price])

        for i in range(1, max_cnt):
            if self.direction == 'buy':
                self.price_lists = np.append(
                    self.price_lists, self.price_lists[-1] - (self.price_lists[-1] * margin_call[i]))
            else:
                self.price_lists = np.append(
                    self.price_lists, self.price_lists[-1] + (self.price_lists[-1] * margin_call[i]))

    def is_add_open(self, price):  # 是否加仓
        if self.curr >= self.max_cnt:
            return []
        if self.direction == 'buy':
            return self.price_lists[self.price_lists >= price]
        else:
            return self.price_lists[self.price_lists <= price]

    def is_close_all(self, price):  # 是否平仓
        if self.curr == 0:
            return False

        result = 0
        for i in range(0, self.curr):
            result += self.price_lists[i] * self.bs[i]

        avg_price = result / sum(self.bs[:self.curr])

        if self.direction == 'buy':
            return price > (avg_price + (avg_price * self.close_call[self.curr - 1]))
        else:
            return price < (avg_price - (avg_price * self.close_call[self.curr - 1]))

    def curr_open(self):  # 当前开仓
        if self.curr == 0:
            return 0
        return sum(self.bs[:self.curr])


mas = Martingale(open_price=2000, max_cnt=5, direction='buy',
                 margin_call=0.01, close_call=0.005)

print(mas.is_add_open(1960))


[2000.  1980.  1960.2]


In [31]:
import numpy as np


max_cnt = 5
direction = 'buy'
margin_call = np.linspace(0, 0.01, num=5).tolist()
close_call = np.linspace(0.02, 0.01, num=5).tolist()
bs = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192]
price_lists = np.array([2000])
close_lists = []
order_lists = []

for i in range(1, max_cnt):
    if direction == 'buy':
        price_lists = np.append(
            price_lists, price_lists[-1] - (price_lists[-1] * margin_call[i]))
    else:
        price_lists = np.append(
            price_lists, price_lists[-1] + (price_lists[-1] * margin_call[i]))


result = 0
for i in range(0, max_cnt):
    result += price_lists[i] * bs[i]
    avg = result / sum(bs[:i+1])
    if direction == 'buy':
        close_lists.append(avg + (avg * close_call[i]))
    else:
        close_lists.append(avg - (avg * close_call[i]))
    
print(price_lists)
print(close_lists)


for item in price_lists.round(2):
    print(item) 





[2000.         1995.         1985.025      1970.1373125  1950.43593938]
[2040.0, 2031.6083333333333, 2019.8645, 2004.1558987499998, 1984.1017827516127]
2000.0
1995.0
1985.02
1970.14
1950.44


In [1]:
import time
from threading import Timer
from huobi.linear_swap.rest import account, market, order
from config.linairx001 import ACCESS_KEY, SECRET_KEY

symbol = 'HT-USDT'

client = market.Market()
orderClient = order.Order(ACCESS_KEY,
                          SECRET_KEY)
accountClient = account.Account(ACCESS_KEY, SECRET_KEY)

result = accountClient.get_balance_valuation({"valuation_asset": 'USD'})
# result = accountClient.get_account_info({"contract_code": 'XRP-USD'})
# result = accountClient.get_position_info({"contract_code": 'XRP-USD'})

print(result)

# 价格精度 数值
result = client.get_contract_info({"contract_code": symbol})
price_tick = str(result.get('data')[0].get('price_tick'))

if '1e-' in price_tick:
    print(int(price_tick.split('-')[1]))
else:
    print(len(price_tick.split('.')[1]))


def fetchKLines(symbol: str, interval: str, limit: str):
    return client.get_kline(
        {"contract_code": symbol, "period": interval, "size": limit})

klines = fetchKLines(symbol=symbol, interval='1min', limit='1')

print(klines)


result = orderClient.cross_order(data={
    "contract_code": symbol,
    "volume": 1,
    "direction": 'buy',
    "offset": 'open',
    "price": '6.62',
    "lever_rate": 10,
    "order_price_type": 'limit'
})

print(result)


{'status': 'ok', 'data': [{'valuation_asset': 'USD', 'balance': '129.964784306078464317'}], 'ts': 1665980527214}
4
{'ch': 'market.HT-USDT.kline.1min', 'ts': 1665980527420, 'status': 'ok', 'data': [{'id': 1665980520, 'open': 6.6644, 'close': 6.6683, 'low': 6.6644, 'high': 6.6689, 'amount': 77.2, 'vol': 772, 'trade_turnover': 514.7856, 'count': 8}]}
{'status': 'ok', 'data': {'order_id': 1031542627511095296, 'order_id_str': '1031542627511095296'}, 'ts': 1665980527538}


In [53]:
result = orderClient.cross_cancel({
    "order_id": '1031209124452663299',
    "contract_code": 'CLV-USDT'
})

print(result)
result = orderClient.cross_get_order_info({
    "order_id": '1031209124452663299',
    "contract_code": 'CLV-USDT'
})

print(result)

{'status': 'ok', 'data': {'errors': [{'order_id': '1031209124452663299', 'err_code': 1071, 'err_msg': 'Repeated cancellation. Your order has been canceled.'}], 'successes': ''}, 'ts': 1665906508940}
{'status': 'ok', 'data': [{'business_type': 'swap', 'contract_type': 'swap', 'pair': 'CLV-USDT', 'symbol': 'CLV', 'contract_code': 'CLV-USDT', 'volume': 1, 'price': 0.09, 'order_price_type': 'limit', 'order_type': 1, 'direction': 'buy', 'offset': 'open', 'lever_rate': 10, 'order_id': 1031209124452663299, 'client_order_id': None, 'created_at': 1665901014203, 'trade_volume': 0, 'trade_turnover': 0, 'fee': 0, 'trade_avg_price': None, 'margin_frozen': 0.0, 'profit': 0, 'status': 7, 'order_source': 'api', 'order_id_str': '1031209124452663299', 'fee_asset': 'USDT', 'liquidation_type': '0', 'canceled_at': 1665901034528, 'margin_asset': 'USDT', 'margin_account': 'USDT', 'margin_mode': 'cross', 'is_tpsl': 0, 'real_profit': 0, 'trade_partition': 'USDT', 'reduce_only': 0}], 'ts': 1665906509054}


In [52]:


np.linspace(0.01, 0, num=5)

# [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192][:5]

array([0.01  , 0.0075, 0.005 , 0.0025, 0.    ])

In [2]:
import numpy as np

np.linspace(0.01, -0.01, num=5).tolist()

[0.01, 0.005, 0.0, -0.004999999999999999, -0.01]

In [8]:
import numpy as np
print(np.linspace(0, 0.05, num=10).tolist())
print(np.linspace(0.5, 0.1, num=10).tolist())

[0.0, 0.005555555555555556, 0.011111111111111112, 0.016666666666666666, 0.022222222222222223, 0.02777777777777778, 0.03333333333333333, 0.03888888888888889, 0.044444444444444446, 0.05]
[0.5, 0.45555555555555555, 0.4111111111111111, 0.3666666666666667, 0.3222222222222222, 0.2777777777777778, 0.23333333333333334, 0.18888888888888888, 0.14444444444444443, 0.1]


In [4]:
[item for item in "123.3,2,2,2".split(',')]

['123.3', '2', '2', '2']

In [58]:
curr = 0
ss = 1

def a():
    global curr, ss 
    curr = 1
    ss = 2

a()
print(curr, ss)

1 2
